In [ ]:
pip install earthengine-api

In [ ]:
pip install earthengine-api --upgrade

In [ ]:
pip install earthpy

In [ ]:
pip install geehydro

In [21]:
import ee
import datetime

import earthpy as ep

from glob import glob
import geehydro
import pandas as pd

import folium

import earthpy.spatial as es

import rasterio as rio
from rasterio.plot import plotting_extent
from rasterio.plot import show
from rasterio.plot import reshape_as_raster, reshape_as_image

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

import plotly.graph_objects as go

from IPython.display import Image

In [6]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=SY2sSNCrY9oketuzmqNT15AkvY9rO40JB7E_JPpFUpE&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgAIIM-0_PQiTVzIfrIHDkVcbPklpzJa1hk4asHNjSFHzhsWj7VEkA

Successfully saved authorization token.


In [7]:
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")

## Case Study 1: Ituna/Itata Indigenous Land in Brazil

In [8]:
# the Ituna/Itatá Indigenous Land in Brazil.
Ituna_map = folium.Map(location=[-4.06738, -52.034], zoom_start= 15)
Ituna_map

In [9]:
# setting the Area of Interest (AOI)
Ituna_AOI = ee.Geometry.Rectangle([-51.84448, -3.92180,
                                   -52.23999, -4.38201])

In [10]:
# filter area
# Landsat 8 available from 2013-03-18 forwards
landsat_AOI = landsat.filterBounds(Ituna_AOI).filterDate('2013-03-28','2021-12-31')

In [11]:
# Check how many images you have
print('Total number:', landsat_AOI.size().getInfo())

Total number: 116


In [12]:
# Names of Landsat 8 Bands
landsat_AOI.first().bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B10',
 'B11',
 'sr_aerosol',
 'pixel_qa',
 'radsat_qa']

In [13]:
from datetime import datetime as dt

# choose the least cloudy image as example
least_cloudy = ee.Image(landsat_AOI.sort('CLOUD_COVER').first())


# how cloudy is it?
print('Cloud Cover (%):', least_cloudy.get('CLOUD_COVER').getInfo())

# when was this image taken?
date = ee.Date(least_cloudy.get('system:time_start'))

time = date.getInfo()['value']/1000

dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')

Cloud Cover (%): 0


'2017-07-20 13:36:07'

In [14]:
parameters = {'min': 0,
              'max': 1000,
              'dimensions': 512,
              'bands': ['B4', 'B3', 'B2'],
              'region': Ituna_AOI}


imgdown = least_cloudy.toUint16()
             
Image(url = imgdown.getThumbUrl(parameters))

In [15]:
## If you wanted to export an image
'''
task = ee.batch.Export.image.toDrive(image=imgdown,
                                     description='Brazil Image',
                                     scale=30,
                                     fileNamePrefix='my_export_brazil',
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF')
task.start()
'''

"\ntask = ee.batch.Export.image.toDrive(image=imgdown,\n                                     description='Brazil Image',\n                                     scale=30,\n                                     fileNamePrefix='my_export_brazil',\n                                     crs='EPSG:4326',\n                                     fileFormat='GeoTIFF')\ntask.start()\n"

In [16]:
# Check the status of downloading the image
'''
task.status()
'''

'\ntask.status()\n'

### Running NDVI on the least cloudy image

In [17]:
ndvi = least_cloudy.normalizedDifference(['B5', 'B4'])

In [18]:
# Visualize NDVI results
palette = ['red', 'yellow', 'green']
ndvi_parameters = {'min': 0,
                   'max': 1,
                   'dimensions': 512,
                   'palette': palette,
                   'region': Ituna_AOI}
Ituna_map.addLayer(ndvi, ndvi_parameters)
Ituna_map

##### Obtaining Yearly Average NDVI Results for this Area

In [23]:
#NDVI calculation:
def ndvi_func(i):
    ndvi = i.normalizedDifference (['B5', 'B4']).rename('NDVI')
    return i.addBands(ndvi)

images_ndvi = landsat_AOI.map(ndvi_func)

#Calculating year wise NDVI.  Result is a list of dictionaries.
years = ee.List.sequence(2013,2021)
def year_func(y):
    range = ee.Filter.calendarRange (y, y, 'year')
    mean = (images_ndvi
            .filter(range)
            .select('NDVI')
            .mean()
            .reduceRegion(ee.Reducer.mean(), Ituna_AOI, 30))
    return mean.set('year', y)

yearwise_ndvi = years.map(year_func).getInfo()
df = pd.DataFrame(yearwise_ndvi)
print(df)

       NDVI  year
0  0.320958  2013
1  0.518476  2014
2  0.742392  2015
3  0.736439  2016
4  0.713168  2017
5  0.723214  2018
6  0.816954  2019
7  0.650084  2020
8  0.559372  2021


## Case Study 2: Seattle area

In [46]:
# the Seattle area in USA.
Seattle_map = folium.Map(location=[47.6062, -122.3321], zoom_start=12)#47.608013, -122.335167], zoom_start= 12)
Seattle_map

In [47]:
# setting the Area of Interest (AOI)
Seattle_AOI = ee.Geometry.Rectangle([-122.445465, 47.712397, 
                                   -122.253510, 47.542795])

In [48]:
# filter area
landsat_AOI = landsat.filterBounds(Seattle_AOI).filterDate('2013-01-01','2021-12-31')

In [49]:
# Check how many images you have
print('Total number:', landsat_AOI.size().getInfo())

Total number: 320


In [50]:
from datetime import datetime as dt

# the least cloudy image
least_cloudy = ee.Image(landsat_AOI.sort('CLOUD_COVER').first())


# how cloudy is it?
print('Cloud Cover (%):', least_cloudy.get('CLOUD_COVER').getInfo())

# when was this image taken?
date = ee.Date(least_cloudy.get('system:time_start'))

time = date.getInfo()['value']/1000

dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')

Cloud Cover (%): 0.06


'2016-08-19 19:01:56'

In [51]:
parameters = {'min': 0,
              'max': 1000,
              'dimensions': 512,
              'bands': ['B4', 'B3', 'B2'],
              'region': Seattle_AOI}


imgdown = least_cloudy.toUint16()
             
Image(url = imgdown.getThumbUrl(parameters))

##### NDVI on Seattle

In [52]:
ndvi = least_cloudy.normalizedDifference(['B5', 'B4'])

In [53]:
palette = ['red', 'yellow', 'green']
ndvi_parameters = {'min': 0,
                   'max': 1,
                   'dimensions': 512,
                   'palette': palette,
                   'region': Seattle_AOI}
Seattle_map.addLayer(ndvi, ndvi_parameters)
Seattle_map

In [56]:
#NDVI calculation:
def ndvi_func(i):
    ndvi = i.normalizedDifference (['B5', 'B4']).rename('NDVI')
    return i.addBands(ndvi)

images_ndvi = landsat_AOI.map(ndvi_func)

#Calculating year wise NDVI.  Result is a list of dictionaries.
years = ee.List.sequence(2013,2021)
def year_func(y):
    range = ee.Filter.calendarRange (y, y, 'year')
    mean = (images_ndvi
            .filter(range)
            .select('NDVI')
            .mean()
            .reduceRegion(ee.Reducer.mean(), Seattle_AOI, 30))
    return mean.set('year', y)

yearwise_ndvi = years.map(year_func).getInfo()
df = pd.DataFrame(yearwise_ndvi)
print(df)

       NDVI  year
0  0.135939  2013
1  0.104162  2014
2  0.157379  2015
3  0.110165  2016
4  0.111646  2017
5  0.112831  2018
6  0.142973  2019
7  0.135991  2020
8  0.127688  2021


# Trying it with another older dataset

In [59]:
older_landsat = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")

In [57]:
# the Seattle area in USA.
Seattle_map = folium.Map(location=[47.6062, -122.3321], zoom_start=12)#47.608013, -122.335167], zoom_start= 12)
Seattle_map

In [60]:
# setting the Area of Interest (AOI)
Seattle_AOI = ee.Geometry.Rectangle([-122.445465, 47.712397, 
                                   -122.253510, 47.542795])

In [61]:
# filter area
landsat_AOI = older_landsat.filterBounds(Seattle_AOI).filterDate('1984-01-01','2012-12-31')

In [62]:
# Check how many images you have
print('Total number:', landsat_AOI.size().getInfo())

Total number: 896


In [63]:
from datetime import datetime as dt

# the least cloudy image
least_cloudy = ee.Image(landsat_AOI.sort('CLOUD_COVER').first())


# how cloudy is it?
print('Cloud Cover (%):', least_cloudy.get('CLOUD_COVER').getInfo())

# when was this image taken?
date = ee.Date(least_cloudy.get('system:time_start'))

time = date.getInfo()['value']/1000

dt.utcfromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')

Cloud Cover (%): 0


'1985-08-23 18:25:12'

In [73]:
parameters = {'min': 0,
              'max': 0.5,
              'dimensions': 512,
              'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
              'region': Seattle_AOI}


imgdown = least_cloudy.toUint16()
             
Image(url = imgdown.getThumbUrl(parameters))

In [68]:
ndvi = least_cloudy.normalizedDifference(['SR_B5', 'SR_B4'])

In [69]:
palette = ['red', 'yellow', 'green']
ndvi_parameters = {'min': 0,
                   'max': 1,
                   'dimensions': 512,
                   'palette': palette,
                   'region': Seattle_AOI}
Seattle_map.addLayer(ndvi, ndvi_parameters)
Seattle_map

In [72]:
#NDVI calculation:
def ndvi_func(i):
    ndvi = i.normalizedDifference (['SR_B5', 'SR_B4']).rename('NDVI')
    return i.addBands(ndvi)

images_ndvi = landsat_AOI.map(ndvi_func)

#Calculating year wise NDVI.  Result is a list of dictionaries.
years = ee.List.sequence(1984,2012)
def year_func(y):
    range = ee.Filter.calendarRange (y, y, 'year')
    mean = (images_ndvi
            .filter(range)
            .select('NDVI')
            .mean()
            .reduceRegion(ee.Reducer.mean(), Seattle_AOI, 30))
    return mean.set('year', y)

yearwise_ndvi = years.map(year_func).getInfo()
df = pd.DataFrame(yearwise_ndvi)
print(df)

        NDVI  year
0  -0.079930  1984
1  -0.052612  1985
2  -0.081877  1986
3  -0.070593  1987
4  -0.068597  1988
5  -0.061402  1989
6  -0.058006  1990
7  -0.036185  1991
8  -0.087773  1992
9  -0.073947  1993
10 -0.070807  1994
11 -0.065419  1995
12 -0.077736  1996
13 -0.087701  1997
14 -0.078417  1998
15 -0.058972  1999
16 -0.072067  2000
17 -0.075601  2001
18 -0.076871  2002
19 -0.092960  2003
20 -0.064001  2004
21 -0.063825  2005
22 -0.068553  2006
23 -0.062510  2007
24 -0.074125  2008
25 -0.089746  2009
26 -0.065729  2010
27 -0.086765  2011
28       NaN  2012
